In [1]:
import lightgbm as lgb
import sys
import logging
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna

# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

SEED = 55

np.random.seed(SEED)

/home/barauna/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:369: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/barauna/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:369: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/barauna/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:369: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [ ]:
import optuna

# Specify the path to your .db file
db_path = "path_to_your_study.db"

# Load the study from the .db file
study = optuna.load_study(study_name="your_study_name", storage=f"sqlite:///{db_path}")

# Now you can work with the loaded study


In [20]:
def objective(trial):
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

    param = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
    gbm = lgb.train(param, dtrain, valid_sets=[dvalid], callbacks=[pruning_callback])

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [21]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
)
study.optimize(objective, n_trials=100, timeout=600)




[I 2023-08-04 15:41:34,758] A new study created in memory with name: no-name-8a3d5dbf-4e75-4281-8c2e-4a543a760399
[I 2023-08-04 15:41:35,159] Trial 0 finished with value: 0.972027972027972 and parameters: {'bagging_fraction': 0.6247240713084175, 'bagging_freq': 7, 'min_child_samples': 75}. Best is trial 0 with value: 0.972027972027972.
[I 2023-08-04 15:41:35,334] Trial 1 finished with value: 0.972027972027972 and parameters: {'bagging_fraction': 0.759195090518222, 'bagging_freq': 2, 'min_child_samples': 19}. Best is trial 0 with value: 0.972027972027972.
[I 2023-08-04 15:41:35,375] Trial 2 finished with value: 0.958041958041958 and parameters: {'bagging_fraction': 0.4348501673009197, 'bagging_freq': 7, 'min_child_samples': 62}. Best is trial 0 with value: 0.972027972027972.
[I 2023-08-04 15:41:35,441] Trial 3 finished with value: 0.9790209790209791 and parameters: {'bagging_fraction': 0.8248435466776274, 'bagging_freq': 1, 'min_child_samples': 98}. Best is trial 3 with value: 0.9790209

In [4]:
plot_optimization_history(study)

In [6]:
plot_contour(study, params=["bagging_freq", "bagging_fraction","min_child_samples"])

In [22]:
# Save study in a database file and restore it: 
# Path: Optuna_dashboard.ipynb
study_name = "example-study"  # Unique identifier of the study.
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
    study_name=study_name,
    storage="sqlite:///example.db",  # Create a SQLite DB file.
    load_if_exists=True,
)


[I 2023-08-04 15:41:49,548] Using an existing study with name 'example-study' instead of creating a new one.


In [25]:
ls

002_multi_objective.ipynb  jornal-CNN-0-study.db   study.db
example.db                 main.py
example-study.db           Optuna_dashboard.ipynb


In [29]:
import optuna
import logging

In [30]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name   = f"jornal-CNN-0-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
if os.path.isfile(study_name+'.db'): #If there is a `jornal-study` file, then load it
    study    = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
    study.optimize(objective, n_trials=number_trials-len(study.trials),n_jobs=number_jobs)

RuntimeError: The runtime optuna version 3.0.3 is no longer compatible with the table schema (set up by optuna 3.0.3). Please execute `$ optuna storage upgrade --storage $STORAGE_URL` for upgrading the storage.

In [34]:
#setup optuna to version 3.0.3
!pip install optuna==3.0.3

optuna 3.2.0
